In [1]:
from __future__ import print_function

In [3]:
import pandas as pd
import scipy.stats as stats
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [71]:
#first look at data, let's look at just the first 2 rows
nov11 = pd.read_csv('/Users/alexandrasmith/Documents/DataScience/Intro_to_Data_Science/irish_EV_stations_data/201611a.txt',sep='\t',names = ["Datetime","Time","ChargePointID","ChargePointType","Status","Latitude_Longitude","Address","Latitude","Longitude"
], nrows=2)

In [72]:
nov11

,Datetime,Time,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude
0,20161102,640,CP:C2YMD-A/B,StandardType2,Part,"-7.204457,53.863850","The Grove, Ballyjamesduff, County Cavan",-7.204457,53.863850
1,20161102,640,CP:C4ZH3,StandardType2,OOS,"-7.361237,53.994044","Off Farnham Street, Cavan Town, County Cavan",-7.361237,53.994044


In [127]:
#load all data with correct headers (I decided the column names based on info about dataset and looking at first 2 rows above)
nov11 = pd.read_csv('/Users/alexandrasmith/Documents/DataScience/Intro_to_Data_Science/irish_EV_stations_data/201611a.txt',sep='\t',names = ["Datetime","Time","ChargePointID","ChargePointType","Status","Latitude_Longitude","Address","Latitude","Longitude"
])

In [128]:
nov11.head()

,Datetime,Time,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude
0,20161102,640,CP:C2YMD-A/B,StandardType2,Part,"-7.204457,53.863850","The Grove, Ballyjamesduff, County Cavan",-7.204457,53.863850
1,20161102,640,CP:C4ZH3,StandardType2,OOS,"-7.361237,53.994044","Off Farnham Street, Cavan Town, County Cavan",-7.361237,53.994044
2,20161102,640,CP:C4547-A/B,StandardType2,OOC,"-7.082769,53.834939","Main Street, Virginia, County Cavan",-7.082769,53.834939
3,20161102,640,CP:C6GPC,StandardType2,Part,"-8.686061,52.358365","The Turrets, Off Main Street, Charleville, Cou...",-8.686061,52.358365
4,20161102,640,CP:C6BTQ,StandardType2,Part,"-8.520562,51.709117","Fire Brigade Car Park, New Road, Kinsale, Coun...",-8.520562,51.709117


In [156]:
nov11.shape

(1213520, 7)

In [152]:
#now i need to add the other months
dec11 = pd.read_csv('/Users/alexandrasmith/Documents/DataScience/Intro_to_Data_Science/irish_EV_stations_data/201612a.txt',sep='\t',names = ["Datetime","Time","ChargePointID","ChargePointType","Status","Latitude_Longitude","Address","Latitude","Longitude"
])

In [153]:
dec11.head()

,Datetime,Time,ChargePointID,ChargePointType,Status,Latitude_Longitude,Address,Latitude,Longitude
0,20161201,0,CP:C67D2,StandardType2,OOC,"-6.955983,52.702794","The Parade, Bagenalstown, County Carlow",-6.955983,52.702794
1,20161201,0,CP:C4GTM,StandardType2,OOC,"-6.736986,52.801581","Inner Relief Road, Tullow, County Carlow",-6.736986,52.801581
2,20161201,0,CP:C4ZH3,StandardType2,OOS,"-7.361237,53.994044","Off Farnham Street, Cavan Town, County Cavan",-7.361237,53.994044
3,20161201,0,CP:C4W54,StandardType2,OOC,"-9.647069,52.683379","Public Car Park, East End, Kilkee, County Clare",-9.647069,52.683379
4,20161201,0,CP:C6WY4,StandardType2,OOC,"-9.290289,53.027069","Hydro Hotel, Main Street, Lisdoonvarna, County...",-9.290289,53.027069


In [157]:
dec11.shape

(1085735, 9)

In [155]:
nov11.append(dec11,ignore_index=True)

,Address,ChargePointID,ChargePointType,Datetime,Latitude,Latitude_Longitude,Longitude,Status,Time
0,"The Grove, Ballyjamesduff, County Cavan",CP:C2YMD-A/B,StandardType2,20161102640,-7.204457,NaN,53.863850,Part,NaN
1,"Off Farnham Street, Cavan Town, County Cavan",CP:C4ZH3,StandardType2,20161102640,-7.361237,NaN,53.994044,OOS,NaN
2,"Main Street, Virginia, County Cavan",CP:C4547-A/B,StandardType2,20161102640,-7.082769,NaN,53.834939,OOC,NaN
3,"The Turrets, Off Main Street, Charleville, Cou...",CP:C6GPC,StandardType2,20161102640,-8.686061,NaN,52.358365,Part,NaN
4,"Fire Brigade Car Park, New Road, Kinsale, Coun...",CP:C6BTQ,StandardType2,20161102640,-8.520562,NaN,51.709117,Part,NaN
5,"Navenny Street, Ballybofey, County Donegal",CP:C7GRZ,StandardType2,20161102640,-7.783045,NaN,54.797965,OOC,NaN
6,"Market Yard Car Park, Main Street, Ballyshanno...",CP:C7DGY,StandardType2,20161102640,-8.188621,NaN,54.501145,OOC,NaN
7,"The Aura Lesiure Centre, Ballymacool Street, L...",CP:C9HRF,StandardType2,20161102640,-7.751151,NaN,54.946769,OOS,NaN
8,"Hospital Car Park, Kilmacrennan Road, Letterke...",CP:C75HT,StandardType2,20161102640,-7.731468,NaN,54.961006,OOS,NaN
9,"Justice Walsh Car Park, Off Bank Lane, Letterk...",CP:C7FDW,StandardType2,20161102640,-7.734554,NaN,54.951508,OOC,NaN


In [129]:
del nov11['Latitude_Longitude'] #delete duplicate column of latitude and longitude

In [130]:
nov11.head(10)

,Datetime,Time,ChargePointID,ChargePointType,Status,Address,Latitude,Longitude
0,20161102,640,CP:C2YMD-A/B,StandardType2,Part,"The Grove, Ballyjamesduff, County Cavan",-7.204457,53.863850
1,20161102,640,CP:C4ZH3,StandardType2,OOS,"Off Farnham Street, Cavan Town, County Cavan",-7.361237,53.994044
2,20161102,640,CP:C4547-A/B,StandardType2,OOC,"Main Street, Virginia, County Cavan",-7.082769,53.834939
3,20161102,640,CP:C6GPC,StandardType2,Part,"The Turrets, Off Main Street, Charleville, Cou...",-8.686061,52.358365
4,20161102,640,CP:C6BTQ,StandardType2,Part,"Fire Brigade Car Park, New Road, Kinsale, Coun...",-8.520562,51.709117
5,20161102,640,CP:C7GRZ,StandardType2,OOC,"Navenny Street, Ballybofey, County Donegal",-7.783045,54.797965
6,20161102,640,CP:C7DGY,StandardType2,OOC,"Market Yard Car Park, Main Street, Ballyshanno...",-8.188621,54.501145
7,20161102,640,CP:C9HRF,StandardType2,OOS,"The Aura Lesiure Centre, Ballymacool Street, L...",-7.751151,54.946769
8,20161102,640,CP:C75HT,StandardType2,OOS,"Hospital Car Park, Kilmacrennan Road, Letterke...",-7.731468,54.961006
9,20161102,640,CP:C7FDW,StandardType2,OOC,"Justice Walsh Car Park, Off Bank Lane, Letterk...",-7.734554,54.951508


In [131]:
nov11.dtypes #check types of columns

Datetime             int64
Time                 int64
ChargePointID       object
ChargePointType     object
Status              object
Address             object
Latitude           float64
Longitude          float64
dtype: object

In [132]:
nov11.Datetime = nov11.Datetime.astype(str) #convert Datetime to str to combine date (Datetime) and time (Time)
nov11.Time = nov11.Time.astype(str) #convert Time to str to combine date (Datetime) and time (Time)

In [140]:
nov11.dtypes

Datetime            object
ChargePointID       object
ChargePointType     object
Status              object
Address             object
Latitude           float64
Longitude          float64
dtype: object

In [134]:
nov11['Datetime'] = nov11['Datetime'].str.cat(nov11['Time'].values.astype(str), sep='') #add Time to Datetime column

In [135]:
nov11.head()

,Datetime,Time,ChargePointID,ChargePointType,Status,Address,Latitude,Longitude
0,20161102640,640,CP:C2YMD-A/B,StandardType2,Part,"The Grove, Ballyjamesduff, County Cavan",-7.204457,53.863850
1,20161102640,640,CP:C4ZH3,StandardType2,OOS,"Off Farnham Street, Cavan Town, County Cavan",-7.361237,53.994044
2,20161102640,640,CP:C4547-A/B,StandardType2,OOC,"Main Street, Virginia, County Cavan",-7.082769,53.834939
3,20161102640,640,CP:C6GPC,StandardType2,Part,"The Turrets, Off Main Street, Charleville, Cou...",-8.686061,52.358365
4,20161102640,640,CP:C6BTQ,StandardType2,Part,"Fire Brigade Car Park, New Road, Kinsale, Coun...",-8.520562,51.709117


In [136]:
#delete duplicate time column
del nov11['Time']

In [137]:
nov11.head()

,Datetime,ChargePointID,ChargePointType,Status,Address,Latitude,Longitude
0,20161102640,CP:C2YMD-A/B,StandardType2,Part,"The Grove, Ballyjamesduff, County Cavan",-7.204457,53.863850
1,20161102640,CP:C4ZH3,StandardType2,OOS,"Off Farnham Street, Cavan Town, County Cavan",-7.361237,53.994044
2,20161102640,CP:C4547-A/B,StandardType2,OOC,"Main Street, Virginia, County Cavan",-7.082769,53.834939
3,20161102640,CP:C6GPC,StandardType2,Part,"The Turrets, Off Main Street, Charleville, Cou...",-8.686061,52.358365
4,20161102640,CP:C6BTQ,StandardType2,Part,"Fire Brigade Car Park, New Road, Kinsale, Coun...",-8.520562,51.709117


In [147]:
nov11.dtypes

Datetime            object
ChargePointID       object
ChargePointType     object
Status              object
Address             object
Latitude           float64
Longitude          float64
dtype: object

In [142]:
#from datetime import datetime

#DateFormat = '%y%m%d%H%M'
#datetime.strptime(nov11.Datetime,DateFormat)

In [151]:
nov11['FormattedDatetime'] = pd.to_datetime(nov11.Datetime,yearfirst=True)
#nov11['Datetime'] = nov11['Datetime'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H%M"))

OverflowError: signed integer is greater than maximum